In [1]:
import scipy.io

import sys
import numpy as np
import matplotlib.pyplot as plt
import gpflow
import tensorflow as tf
from gpflow.ci_utils import ci_niter
sys.path.append('../code')
from gpar.regression import GPARRegressor
from gplar_q import GPLARmodel


In [2]:
import pandas as pd
london = pd.read_csv('london.csv')

In [3]:
london

,Unnamed: 0,price,latitude,longitude,type_D,type_F,type_S,type_T
0,0,13.038982,51.510670,-0.052594,0,1,0,0
1,1,12.721886,51.511312,-0.053388,0,1,0,0
2,2,11.461632,51.513079,-0.054667,0,1,0,0
3,3,12.873902,51.510175,-0.061262,0,1,0,0
4,4,12.734247,51.510214,-0.054933,0,1,0,0
...,...,...,...,...,...,...,...,...
33721,33721,14.417091,51.512084,-0.118208,0,1,0,0
33722,33722,14.533350,51.511471,-0.118233,0,1,0,0
33723,33723,14.745896,51.512158,-0.112210,0,1,0,0
33724,33724,14.829103,51.511505,-0.113562,0,1,0,0


In [4]:
ntrain = 20000

train_index = np.random.choice(range(len(london.index)),ntrain)
test_index = np.setdiff1d(range(len(london.index)),train_index)

x_train = np.array(london[['latitude','longitude']])[train_index]
y_train = np.array(london[['price','type_F','type_S','type_T']])[train_index]
x_test = np.array(london[['latitude','longitude']])[test_index]
y_test = np.array(london[['price','type_F','type_S','type_T']])[test_index]

In [5]:
invlink = gpflow.likelihoods.RobustMax(10)  # Robustmax inverse link function
likelihoods = [['Multiclass',[1,2,3],gpflow.likelihoods.MultiClass(4, invlink=invlink)],
                ['Gaussian',0,gpflow.likelihoods.Gaussian(0.01)]]

In [6]:
from scipy.cluster.vq import kmeans2

M = 200
x_ind = kmeans2(x_train, M, minit='points')[0] 

gpar = GPARRegressor(scale=0.1,
                linear=False, linear_scale=1.,
                nonlinear=True, nonlinear_scale=1.,
                noise=0.01,
                impute=True, replace=True, normalise_y=True, x_ind = x_ind)

gplar = GPLARmodel(x_train, y_train, M, gpar, white=False,likelihoods=likelihoods,
                   scale=0.1, linear=False, linear_scale=1.0,
                   nonlinear=True, nonlinear_scale=1.0,
                   noise_inner=1e-5, noise_obs=0.01,
                   minibatch_size=500,
                   num_samples=1)

Training conditionals (total: 5): 1 2 3 4 5
Sampling (total: 50): 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50
Sampling (total: 50): 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50
Sampling (total: 50): 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50


In [ ]:
@tf.function(autograph=False)
def optimization_step(optimizer, model, data):
    with tf.GradientTape(watch_accessed_variables=False) as tape:
        tape.watch(model.trainable_variables)
        objective = -model.maximum_log_likelihood_objective(*data)
        grads = tape.gradient(objective, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    return objective

minibatch_size = 500
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).repeat().shuffle(10000)

from tqdm import tqdm_notebook as tqdm
def run_adam(model, iterations):
    adam = tf.optimizers.Adam(0.01)
    train_it = iter(train_ds.batch(minibatch_size))
    for step in tqdm(range(iterations)):
        neg_elbo, gradient = optimization_step(adam, model, next(train_it))
        if step % 5000 == 0:
            print(elbo.numpy())

maxiter = ci_niter(100000)
run_adam(gplar, maxiter)

In [10]:
mean, var = tf.concat(Hmean[1:4],axis=-1),tf.concat(Hvar[1:4],axis=-1)

In [11]:
#assess accuracy
m, v = gplar.predict_f(x_test, S)
a = (mode(np.argmax(m[:,:3], 2), 0)[0].reshape(y_test[:,:3].shape).astype(int)==y_test[:,:3].astype(int))

#assess smse
pred = np.mean(m[:,3],axis=0)
pred = pred*stds + means
data_mean = np.mean(y_test[:,3])
smse = np.sum((pred-y_test[:,3])**2)/np.sum((data_mean-Ynew[:,3])**2)

#assess likelihood
gplar.likelihoods[0][2].predict_log_density(m[:,:3],v[:,:3],y_test[:,:3])
gplar.likelihoods[1][2].predict_log_density(m[:,3],v[:,3],y_test[:,3])


TensorShape([1, 500, 4])